In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [ ]:
# get dataset
dataset = pd.read_csv("Data/Apache_AllNumeric.csv")
dataset = dataset.sample(frac=1)
dataset_features = dataset.copy()
dataset_labels = dataset_features.pop('PERF')

In [ ]:
# normalize dataset (MinMaxScale)
dataset_features /= dataset_features.max()
dataset_labels /= dataset_labels.max()

In [ ]:
# split dataset train (2/3) test (1/3)
x_train, x_test, y_train, y_test = train_test_split(dataset_features, dataset_labels, test_size=0.33)

In [ ]:
d